In [4]:
%pip install lightning

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\alexa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
# Module importation
from sklearn.model_selection import train_test_split
import sklearn.utils as sk_utilis
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import torch
import os

In [6]:
cwd = os.getcwd()
# select dataset path
data_folder = "datasets"
data_path = os.path.join(cwd, data_folder, "titanic.csv")

# verification
if os.path.exists(data_path):
    print("All is good, we can go on !")
else:
    raise FileExistsError("The path doesn't exist !")

All is good, we can go on !


In [7]:
# load the dataset
data = pd.read_csv(data_path, sep=";")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [8]:
# select just few columns
data_ = data[["pclass", "survived", "sex", "age", "sibsp", "parch"]]

# drop rows with NA value and quantify the gender (?!)
Data  = data_.dropna(axis=0).replace(to_replace=["male","female"], value=[0,1])
Data.describe()

,pclass,survived,sex,age,sibsp,parch
count,1046.000000,1046.000000,1046.000000,1046.000000,1046.000000,1046.00000
mean,2.207457,0.408222,0.370937,29.881135,0.502868,0.42065
std,0.841497,0.491740,0.483287,14.413500,0.912167,0.83975
min,1.000000,0.000000,0.000000,0.166700,0.000000,0.00000
25%,1.000000,0.000000,0.000000,21.000000,0.000000,0.00000
50%,2.000000,0.000000,0.000000,28.000000,0.000000,0.00000
75%,3.000000,1.000000,1.000000,39.000000,1.000000,1.00000
max,3.000000,1.000000,1.000000,80.000000,8.000000,6.00000


In [9]:
# normalisation of each column
DATA = Data/Data.max()
DATA.describe()

,pclass,survived,sex,age,sibsp,parch
count,1046.000000,1046.000000,1046.000000,1046.000000,1046.000000,1046.000000
mean,0.735819,0.408222,0.370937,0.373514,0.062859,0.070108
std,0.280499,0.491740,0.483287,0.180169,0.114021,0.139958
min,0.333333,0.000000,0.000000,0.002084,0.000000,0.000000
25%,0.333333,0.000000,0.000000,0.262500,0.000000,0.000000
50%,0.666667,0.000000,0.000000,0.350000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,0.487500,0.125000,0.166667
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
# split the dataset
X, Y = DATA[["pclass","sex","age","sibsp","parch"]], DATA["survived"]

## Training preparation

In [11]:
import lightning as L

In [12]:
class LightClassifier(L.LightningModule):
    def __init__(self, N_feature, N_classe):
        super(LightClassifier, self).__init__()

        self.linear = torch.nn.Linear(N_feature, N_classe)
        self.sigmoid = torch.nn.Sigmoid()
    
    def training_step(self, batch, batch_idx):

        x,y = batch

        y_pred = self.sigmoid(self.linear(x))[:,0]

        loss = torch.nn.functional.binary_cross_entropy(y, y_pred)
        self.log("Loss/train", loss)

        return loss
    
    def configure_optimizers(self):
        
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)

        return optimizer

In [13]:
Classifier = LightClassifier(N_classe=2, N_feature=5)

In [14]:
from torch.utils.data import TensorDataset, DataLoader

In [15]:
train_loader = DataLoader(
                    TensorDataset(
                        torch.Tensor(X.to_numpy()),
                        torch.Tensor(Y.to_numpy())
                            ),
                        num_workers=15, persistent_workers=True)

In [16]:
trainer = L.Trainer(limit_train_batches=1000, max_epochs=10)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
trainer.fit(model=Classifier, train_dataloaders=train_loader)


  | Name    | Type    | Params
------------------------------------
0 | linear  | Linear  | 12    
1 | sigmoid | Sigmoid | 0     
------------------------------------
12        Trainable params
0         Non-trainable params
12        Total params
0.000     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
